# Navigating Interactive Fiction with Reinforcement Learning

### Bing Wang 
##### MSDS631: Reinforcement Learning, Final Project<br>

<img src="images/michelle-ding-QAOtKq8ehcw-unsplash.jpg" width="850">

## Research Question

## Overview

**What is interactive fiction?**

**Why is this a reinforcement learning problem?**

**What were the tools used in this analysis?**
Jericho
FrotzEnv
Key methods in FrotEnv

**How was reinforcement learning used in this project?**

### RL problem setup:

States: Text at each step<br>
Actions: Sampled text (for Detective, either cardinal direction or [verb][noun] generally)

Deterministic (same state and action always leads to the same value)<br>
Model-free (don't know all states/available actions)<br>
Fully observable (we know what state we're in)<br>
TD updates (update for each state, not full episode)<br>
Terminal/finite (game will end eventually)<br>
History may or may not matter; Markov property almost holds

## Imports and initializations

We will be training our agent on the game *Detective* by Matt Barringer, a film-noir-esque detective mystery included in [Jericho's set of supported interactive fiction stories.](https://jericho-py.readthedocs.io/en/latest/tutorial_quick.html#acquire-games) I chose this game as an initial story to test out training a reinforcement learning agent because it seemed like a shorter, *terminal* game, with a *limited set of actions* for each stage and the game overall.

Let's read in our story file into Jericho's FrotzEnv environment, and initialize our Agent on that. You can find my Agent class in my repo [here](https://github.com/bingwang32/RL_InteractiveFiction/blob/master/agent.py).

In [1]:
from utils import *
from agent import *

In [2]:
story_file = "z-machine-games-master/jericho-game-suite/detective.z5"

In [3]:
agent = Agent(FrotzEnv(story_file))

## Demo of game

##### Human mode

Here is a human-playable demo of the game. At each step, the demo will print a text description of the story at that step (the state). You will be offered a set of likely valid steps corresponding to the text (as generated by Jericho's natural language processing (NLP) capabilities), and will have the ability to type in one of those options or text of your own choice in the input. *Type in "I quit" to exit the game at any point.*

I have included a short playthrough of my own below; feel free to refresh the cell and do a playthrough of your own!

In [4]:
agent.demo_game("human")

Welcome! Enter "I quit" at any point to exit the game.

[Type "help" for more information about this version]  Detective By Matt Barringer. Ported by Stuart Moore. Stuart_Moore@my-deja.com Release 1 / Serial number 000715 / Inform v6.21 Library 6/10 SD  << Chief's office >> You are standing in the Chief's office. He is telling you "The Mayor was murdered yeaterday night at 12:03 am. I want you to solve it before we get any bad publicity or the FBI has to come in. "Yessir!" You reply. He hands you a sheet of paper. Once you have read it, go north or west.  You can see a piece of white paper here.  [Your score has just gone up by ten points.]
Choose one of the following valid moves, or type in something else: ['east', 'take paper', 'west', 'north'] north

<< Outside >> You are outside in the cold. To the east is a dead end. To the west is the rest of the street. Papers are blowing around. It's amazingly cold for this time of year.
Choose one of the following valid moves, or type in somet

##### Walkthrough mode

In walkthrough mode, you can play through the game making the optimal choices of actions at each state, such that you get the highest possible score.

In [5]:
print(f'Maximum possible score in game: {agent.game.get_max_score()}')

Maximum possible score in game: 360


In [6]:
agent.demo_game("walkthrough")

Iteration 0
State: [Type "help" for more information about this version]  Detective By Matt Barringer. Ported by Stuart Moore. Stuart_Moore@my-deja.com Release 1 / Serial number 000715 / Inform v6.21 Library 6/10 SD  << Chief's office >> You are standing in the Chief's office. He is telling you "The Mayor was murdered yeaterday night at 12:03 am. I want you to solve it before we get any bad publicity or the FBI has to come in. "Yessir!" You reply. He hands you a sheet of paper. Once you have read it, go north or west.  You can see a piece of white paper here.  [Your score has just gone up by ten points.]
Action selected: TAKE PAPER
Total score after step: 20

Iteration 1
State: Taken.  [Your score has just gone up by ten points.]
Action selected: READ PAPER
Total score after step: 20

Iteration 2
State: CONFIDENTIAL: Detective was created by Matt Barringer. He has worked hard on this so you better enjoy it. I did have fun making it though. But I'd REALLY appreciate it if you were kind 

360

In [7]:
print(f'Final score in walkthrough playthrough: {agent.game.get_score()}')

Final score in walkthrough playthrough: 360


##### Random mode

We can also use the agent to demo a playthrough of the game where the action at every state is chosen randomly from the set of available actions. We will use this functionality below to simulate random performance over many iterations, and calculate a baseline level of performance for the agent to beat.

In [8]:
agent.demo_game("random")

Iteration 0
State: [Type "help" for more information about this version]  Detective By Matt Barringer. Ported by Stuart Moore. Stuart_Moore@my-deja.com Release 1 / Serial number 000715 / Inform v6.21 Library 6/10 SD  << Chief's office >> You are standing in the Chief's office. He is telling you "The Mayor was murdered yeaterday night at 12:03 am. I want you to solve it before we get any bad publicity or the FBI has to come in. "Yessir!" You reply. He hands you a sheet of paper. Once you have read it, go north or west.  You can see a piece of white paper here.  [Your score has just gone up by ten points.]
Action selected: east
Total score after step: 10

Iteration 1
State: You can't go east from here!  << Chief's office >> You are standing in the Chief's office. He is telling you "The Mayor was murdered yeaterday night at 12:03 am. I want you to solve it before we get any bad publicity or the FBI has to come in. "Yessir!" You reply. He hands you a sheet of paper. Once you have read it, 

Iteration 30
State: << Hallway >> You are in the hallway. To the north is more hallway, and to the east is a door marked "Guests".
Action selected: north
Total score after step: 40

Iteration 31
State: << Hallway >> You are STILL in the hallway. There is EVEN MORE hallway to the north, and a room to the west and a room to the east of you.
Action selected: south
Total score after step: 40

Iteration 32
State: You can't go south from here!  << Hallway >> You are STILL in the hallway. There is EVEN MORE hallway to the north, and a room to the west and a room to the east of you.
Action selected: north
Total score after step: 40

Iteration 33
State: << Hallway >> You are still in the hallway. You can go north to where there is a police officer who will let you outside, or you can go east or west.
Action selected: north
Total score after step: 50

Iteration 34
State: << Outside >> You pass the guard. He nods at you. You are now outside standing on the street. You can go north and east, your 

90

In [9]:
print(f'Final score in random playthrough: {agent.game.get_score()}')

Final score in random playthrough: 90


## Training with Q-learning

Hyperparameters used in the Q-learning implementation were:
- $\epsilon$ = 0.1: The proportion of times the agent selects a random action (exploration) over the best/highest value action (exploitation); explore a fair amount of the time but not too much
- $\alpha$ = 1: Learning rate; update a Q(s,a) value entirely rather than taking a weighted average of previous and new value
- $\gamma$ = 0.999: Discount factor on future rewards; treat current and future rewards almost the same


Let's calculate baseline performance by simulating random play for 1000 iterations and considering its average performance.

In [11]:
baseline = demo_n_games(agent, 1000, 'random')
baseline

62.48

Now, let's see the agent's performance when learning the game over **10, 100, 1000, and 10,000 iterations** to compare their performance to the baseline and to each other.

**10 iterations:**

In [13]:
agent.learn_game(10, 1)

qlearn_10iter = demo_n_games(agent, 1000, 'agent')
qlearn_10iter

Game #1 final score: 70
Game #2 final score: 60
Game #3 final score: 120
Game #4 final score: 60
Game #5 final score: 60
Game #6 final score: 70
Game #7 final score: 90
Game #8 final score: 70
Game #9 final score: 180
Game #10 final score: 220


152.76

**100 iterations:**

In [14]:
agent.learn_game(100, 10)

qlearn_100iter = demo_n_games(agent, 1000, 'agent')
qlearn_100iter

Game #10 final score: 30
Game #20 final score: 160
Game #30 final score: 70
Game #40 final score: 310
Game #50 final score: 290
Game #60 final score: 290
Game #70 final score: 90
Game #80 final score: 320
Game #90 final score: 100
Game #100 final score: 50


216.82

**1000 iterations:**

In [15]:
agent.learn_game(1000, 100)

qlearn_1000iter = demo_n_games(agent, 1000, 'agent')
qlearn_1000iter

Game #100 final score: 70
Game #200 final score: 300
Game #300 final score: 300
Game #400 final score: 320
Game #500 final score: 330
Game #600 final score: 60
Game #700 final score: 60
Game #800 final score: 180
Game #900 final score: 60
Game #1000 final score: 120


225.81

**10000 iterations:**

In [16]:
agent.learn_game(10_000, 1000)

qlearn_10000iter = demo_n_games(agent, 1000, 'agent')
qlearn_10000iter

Game #1000 final score: 60
Game #2000 final score: 90
Game #3000 final score: 30
Game #4000 final score: 80
Game #5000 final score: 310
Game #6000 final score: 330
Game #7000 final score: 300
Game #8000 final score: 40
Game #9000 final score: 290
Game #10000 final score: 80


227.19

## Results and interpretation

Here is a graphical comparison of the performance of 

**Interpretation of results:**
- Q-learning is unstable, as expected, as you can see from the high variance in scores in the training printouts
- Increase in performance starts tapering off at as early as 100 episodes of training. This makes sense because the story of Detective is fairly simple, and there aren't that many "branches" of the story that are heavily dependent on any particular state (e.g. it is possible to get back to a main hallway after exploring a room).
- Since Q-learning saves the reward associated with each state-action combination seen in a Q-table, the agent is effectively learning by exploring the graph structure of the story. Since Detective is relatively simple, the agent can learn to do well simply by exaustively exploring all the states in the game.

## Extensions and lessons learned

**Extensions**
- Tune Q-learning hyperparameters more; try different rates of exploration, etc.
- Try training my agent on other, more complex interactive fiction stories. I wrote my agent to be able to be generalized to other Jericho-compatible stories, so it should be possible.
- Try deep Q-learning (DQN) and other reinforcement learning algorithms.

**Lessons learned**
- Interactive fiction is a unique problem with unique challenges! It requires a model that can accomodate that
- Memoize for performance! Since FrotzEnv's `get_valid_actions()` method required natural language processing (NLP), it was a computationally expensive function, and I minimized calls to it by caching the valid actions for a given state the first time I make a `get_valid_actions()` call on that state.
- Adapting a problem to a custom environment based on [OpenAI's Gym environment](https://gym.openai.com/) can be difficult, especially for an environment where the states are text and not numerical input. I attempted to write a [custom environment for Stable Baselines](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html) in order to apply their [prebuilt reinforcement learning algorithms](https://github.com/hill-a/stable-baselines).

## Conclusion